# 2. Time Series Data Recognition Practice (시계열 데이터 인식 실습)

In [22]:
import numpy as np
import shutil
folderpath= './svc2004'

# 0부터 1,599까지의 값을 가지는 배열(rand_number)을 만들고 순서를 섞음
rand_number = np.array(range(1600))
np.random.shuffle(rand_number)

n_train = 1200
n_test = 400

# 각 서명의 이름이 rand_number의 번호가 되도록 이름을 바꾸어 저장
cnt = 0

for u_id in range(1, 41):
  for s_id in range(1, 41):
    filepath_from = f'{folderpath}/U{u_id}S{s_id}.TXT'
    # 파일의 번호에 따라 학습용/테스트용 데이터로 나눔
    # 학습용 데이터인 경우
    if rand_number[cnt] < n_train:
      filepath_to = f'{folderpath}/train/{rand_number[cnt]}.TXT'
    # 테스트 데이터인 경우
    else:
      filepath_to = f'{folderpath}/test/{rand_number[cnt]}.TXT'
    shutil.move(filepath_from, filepath_to)
    cnt = cnt + 1

In [23]:
# 새롭게 바뀐 서명 데이터의 이름에는 위조 서명여부에 관한 정보가 포함되어 있지 않으므로, 이 정보를 잃어버리지 않기 위해, 각 서명의 진/위조 서명 여부를 numpy 배열에 저장한 후, csv 파일로 저장
# 각 서명의 진/위조 서명 여부를 저장
b_genuine = np.zeros((1600, 2))
b_genuine[:, 0] = range(1600)
cnt = 0
for u_id in range(1, 41):
  for s_id in range(1, 41):
    if s_id < 21:
      # 진 서명
      b_genuine[rand_number[cnt], 1] = 1
    else:
      # 위조 서명
      b_genuine[rand_number[cnt], 1] = 0
    cnt = cnt + 1

In [24]:
# csv 파일로 저장하기 위해서는 pandas 라이브러리를 사용하여 DataFrame 형식으로 변환
import pandas as pd
# 진/위조 서명 여부를 csv 파일로 저장
df_genuine = pd.DataFrame(b_genuine)
df_genuine.columns = ["ID", "bGenuine"]
df_genuine = df_genuine.astype("int32")
df_genuine.to_csv(f"{folderpath}/train_info.csv", index=False)

In [25]:
# 데이터 읽기 및 전처리
import os

# get_max_len 함수
# 파라미터로 학습 및 테스트 폴더의 모든 서명 데이터를 읽고, 전체 서명의 길이 중 가장 긴 길이의 값을 반환
def get_max_len(folderpath_train, folderpath_test):
  n_maxt_len = 0
  for filename in os.listdir(folderpath_train):
    filepath = f'{folderpath_train}/{filename}'
    d, tmp = load_a_sig(filepath)
    if d.shape[0] > n_max_len:
      n_max_len = d.shape[0]

  for filename in os.listdir(folderpath_test):
    filepath = f'{folderpath_test}/{filename}'
    d, tmp = load_a_sig(filepath)
    if d.shape[0] > n_max_len:
      n_max_len = d.shape[0]

  return n_max_len

In [26]:
# load_data 함수
# 파라미터로 주어진 폴더에 들어 있는 서명 파일 전체를 읽고, n_files * n_max_len * 2 크기의 numpy 배열로 만들어 반환
# 파라미터 n_max_len은 전체 서명 데이터들 중 가장 긴 서명의 길이를 의미
# 데이터와 함께 반환하는 len_sign는 각 서명의 실제 길이를 의미
def load_data(folderpath, n_max_len, sig_ids):
  # 폴더에 포함된 파일들의 리스트를 가져옴
  filelist = os.listdir(folderpath)
  # 파일의 갯수를 가져와 n_files 변수에 저장
  n_files = len(filelist)
  # n_files * n_max_len * 2 크기의 numpy 배열을 만듦
  d = np.zeros((n_files, n_max_len, 2))
  # n_files * 1 크기의 배열을 만듦. 이 변수에는 각 서명의 실제 길이를 저장
  len_sign = np.zeros(n_files)

  for i in sig_ids:
    # 파일 이름을 지정
    filepath = f'{folderpath}/{i}.TXT'
    # 서명 데이터를 읽어서 임시변수에 저장
    d_tmp, len_sign[i - sig_ids[0]] = load_a_sig(filepath)
    # 준비한 배열(d)에 저장
    d[i - sig_ids[0], :d_tmp.shape[0], :] = d_tmp
  return d, len_sign

In [27]:
# load_a_sig 함수
# 파라미터로 주어진 서명 파일을 읽고, nPoint * 2 크기의 numpy 배열로 만들어 반환
def load_a_sig(filepath):
  f = open(filepath, "rt")
  # 데이터의 첫 번째 줄을 읽어, nPoint 변수에 저장
  nPoints = int(f.readline())
  d = np.zeros((nPoints, 2))
  # 각 포인트의 갯수만큼 반복하면서 각각의 줄을 읽고, X, y 좌표를 분리하여 저장
  for i in range(nPoints):
    line = f.readline()
    toks = line.split(" ")
    d[i, :] = [int(toks[0]), int(toks[1])]
  f.close()
  return d, nPoints

In [28]:
# 위 함수를 사용하여 데이터를 읽음
# 각 데이터의 길이를 체크하고 최대 길이 확인
n_max_len = get_max_len(f'{folderpath}/train', f'{folderpath}/test')
# 학습용 데이터 load
X_train, X_train_len = load_data(f'{folderpath}/train', n_max_len, range(n_train))
# 테스트용 데이터 load
X_test, X_test_len = load_data(f'{folderpath}/test', n_max_len, range(n_train, 1600))

UnboundLocalError: cannot access local variable 'n_max_len' where it is not associated with a value